# 3. Base del sistema RAG

En esta sección se reutiliza el código desarrollado en el tutorial donde se explica cómo construir un sistema RAG básico. No se volverá a explicar cada componente en detalle, ya que esa explicación se encuentra en el tutorial original. El objetivo aquí es establecer la **arquitectura base** que será utilizada como punto de partida para introducir, en secciones posteriores, las técnicas de query rewriting y reranking.

El código base incluye:

- La carga y segmentación de documentos Markdown utilizando `MarkdownHeaderTextSplitter`, generando chunks semánticamente coherentes con metadatos de trazabilidad.
- La creación de un índice FAISS a partir de embeddings generados con modelos de Google y la configuración de un retriever para búsqueda por similitud.

En este punto, el sistema se limita a preparar los documentos y a exponer un mecanismo de recuperación estándar. No se realiza todavía ninguna reformulación de la consulta ni se aplica ningún proceso de reordenamiento de los documentos recuperados.

⚠️ **Almacenamiento de los documentos**: Los documentos deben estar en una carpeta llamada `data` en la raíz del proyecto. Si utiliza otra estructura, ajuste el parámetro `data_folder` en el código.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from pathlib import Path
import os

# Cargar variables desde el archivo .env
load_dotenv()

# Asignar la clave de API a la variable de entorno esperada por el SDK de Google
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Instancia del modelo de embeddings para indexación de los documentos
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Instancia del modelo de lenguaje para generación de respuestas
language_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
def load_documents(data_folder="data"):
    """
    Carga todos los documentos Markdown desde la carpeta especificada, aplicando
    segmentación basada en estructura de documento (segmentación por encabezados)
    para generar chunks semánticamente coherentes, listos para el proceso de
    indexación en base de datos vectorial.

    Args:
        data_folder (str): Ruta a la carpeta que contiene los archivos Markdown a
            cargar y segmentar.

    Returns:
        list: Lista de chunks representados mediante objetos Document con metadatos
            básicos para garantizar control y trazabilidad.
    """
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4")
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=False
    )

    data_path = Path(data_folder)
    if not data_path.exists():
        raise FileNotFoundError(f"La carpeta '{data_path}' no existe")

    markdown_files = list(data_path.glob("*.md"))
    if not markdown_files:
        raise FileNotFoundError(f"No se encontraron archivos Markdown en la carpeta '{data_path}'")

    all_chunks = []

    for file_path in markdown_files:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()

        chunks = markdown_splitter.split_text(content)

        for i, chunk in enumerate(chunks):
            chunk.metadata.update({
                "source_file": file_path.name,
                "chunk_index": i,
                "size": len(chunk.page_content)
            })

        all_chunks.extend(chunks)

    return all_chunks

In [ ]:
def build_faiss_retriever(chunks, embeddings_model, k=3):
    """
    Construye un índice vectorial FAISS a partir de los chunks previamente generados
    y devuelve un objeto retriever configurado para realizar búsquedas por
    similitud semántica.

    Args:
        chunks (list): Lista de objetos Document que representan los fragmentos de texto
            segmentado a partir de los documentos fuente.
        embeddings_model: Modelo de embeddings utilizado para transformar texto en
            representaciones vectoriales.
        k (int): Número de documentos más similares que se recuperarán durante cada consulta.

    Returns:
        BaseRetriever: Objeto retriever listo para ser utilizado dentro del flujo RAG.
    """
    texts = [chunk.page_content for chunk in chunks]
    metadatas = [chunk.metadata for chunk in chunks]

    vectorstore = FAISS.from_texts(
        texts=texts,
        embedding=embeddings_model,
        metadatas=metadatas
    )

    vectorstore.save_local("./faiss_index")

    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": k}
    )

    return retriever

In [ ]:
# Cargar y segmentar los documentos de entrada
chunks = load_documents()
print(f"Se cargaron {len(chunks)} chunks en total.")

In [ ]:
# Construir el índice FAISS y el retriever (ejecutar después de tener chunks)
retriever = build_faiss_retriever(chunks, embeddings_model, k=3)
print("Retriever configurado. Índice guardado en ./faiss_index")